<a href="https://colab.research.google.com/github/princy620/Projects/blob/main/CrewAl_Using_Gemini_Travel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step-1:Install the Packages

In [ ]:
!pip install crewai
!pip install crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 48.0 MB/s 

#Step-2:Import the packages

In [ ]:
import os
from crewai import Agent, Task, Crew, Process, LLM #Crew AI package
from crewai_tools import SerperDevTool       # For Serper Tool

#Step-3:Access the keys

In [ ]:
from google.colab import userdata
gemini_key=userdata.get('gemini_key')
serper_key=userdata.get('serper_key')
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

#Step-4:LLM and Tool

In [ ]:
# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ["GOOGLE_API_KEY"])

#Step-5:Initialize the agents

In [ ]:

# 🧭 Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find historical sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True  #Transfer the task to next agent
)


# 💰 Budget Planner Agent (Ensures trip stays under $1500)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget flights, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=True
)

# 🗺️ Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)


#Step-6:Agent Tasks

In [ ]:
# 📝 Travel Research Task
research_task = Task(
    description="Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)

#Step-7 Result

In [ ]:
# 🚀 Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)
result = crew.kickoff(inputs={'destination': 'Hydrabad', 'budget': '30000 INR'})
print(result)

# Agent: Travel Researcher
## Task: Find the best historical sites, weather forecast, and public transport hotels for Hydrabad.


# Agent: Travel Researcher
## Thought: tool_code
Thought: I need to gather information about historical sites, weather, and hotels near public transport in Hyderabad.  I'll start by searching for historical sites.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"historical sites in Hyderabad India\"}"
## Tool Output: 
{'searchParameters': {'q': 'historical sites in Hyderabad India', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'THE 10 BEST Hyderabad Sights & Historical Landmarks to Visit (2025)', 'link': 'https://www.tripadvisor.com/Attractions-g297586-Activities-c47-Hyderabad_Hyderabad_District_Telangana.html', 'snippet': 'Hyderabad Landmarks ; 1. Golconda Fort · 4,500 ; 2. Calvary Temple · 537 ; 3. Charminar · 4,350 ; 4. Taj Falaknuma Palace · 962 ; 5. Birla Mandir · 3,515.', 'position': 1}, {'